# Notebook: Pipeline dehazing on Video

In [1]:
%load_ext autoreload
%autoreload 2
import random
from cap.dehaze import dehaze as cap_dehaze_img
from dcp.dehaze import dehaze as dcp_dehaze_img
from cap.dehaze import dehaze_video as cap_dehaze_vid
from dcp.dehaze import dehaze_video as dcp_dehaze_vid

from config import config
import os
from glob import glob
from matplotlib import pyplot as plt
import cv2
from sklearn.metrics import mean_squared_error as compute_mse
from skimage.metrics import peak_signal_noise_ratio as compute_psnr
from skimage.metrics import structural_similarity as compute_ssim
from tqdm import tqdm
import timeit
import numpy as np

In [2]:
_REVIDE_PATH = 'D:\\data\\dehaze\\REVIDE_inside\\Train'
_RESULT_PATH = '../result'
_VIDEO_PATH = 'D:\\data\\dehaze'

In [3]:
def fbf_dehaze_with_global_atm(hazy, method, bov, g_atm):
    """
    hazy removal on video using frame-by-frame with global atmospheric
    param:
        hazy: is the hazy frame
        method: is the name of dehazing method
        bov: is the flag << begin of video >>
        g_atm: is the global atmospheric value
    return:
        dehaze_frame: free-hazy frame
        g_atm: is the refresh global atmospheric value
        time: is the processing time
    """
    start = timeit.default_timer()
    if method == 'dcp':
        dehaze_frame, g_atm = dcp_dehaze_vid(hazy, config['dcp'], g_atm, bov)
    else:
        dehaze_frame, g_atm = cap_dehaze_vid(hazy, config['cap'], g_atm, bov)
    
    end = timeit.default_timer()
    time = end - start
    return dehaze_frame, g_atm, time

def fbf_dehaze_without_global_atm(hazy, method):
    """
    hazy removal on video using frame-by-frame without global atmospheric
    param:
        hazy: is the hazy frame
        method: is the name of dehazing method
    return:
        dehaze_frame: free-hazy frame
        time: is the processing time
    """
    start = timeit.default_timer()
    if method == 'dcp':
        dehaze_frame = dcp_dehaze_img(hazy, config['dcp'])
    else:
        dehaze_frame = cap_dehaze_img(hazy, config['cap'])
    
    end = timeit.default_timer()
    time = end - start
    return dehaze_frame, time

In [4]:
def process_video(in_name, out_name, method, vid_fps, vid_size, use_global = True):
    """
    
    """
    cap = cv2.VideoCapture(os.path.join(_VIDEO_PATH,in_name))
    fourcc = cv2.VideoWriter_fourcc(*'XVID')
    out = cv2.VideoWriter(out_name, fourcc, vid_fps, vid_size)
    list_time = []
    while(cap.isOpened()):
        ret, hazy_frame = cap.read()
        try:
            if (cv2.waitKey(1)) & (0xFF == ord('q')) & (hazy_frame is None):
                cap.release()
            else:
                if use_global:
                    if len(list_time)==0:
                        bov = True
                        g_atm = np.zeros((1,3))
                    else:
                        bov = False
                    dehaze_frame, g_atm, time = fbf_dehaze_with_global_atm(hazy_frame, method, bov, g_atm)
                else:
                    dehaze_frame, time = fbf_dehaze_without_global_atm(hazy_frame, method)
                list_time.append(time)
                out.write(dehaze_frame)
        except Exception as e:
#             print(e)
            cap.release()
    out.release()
    return list_time

In [5]:
vid_in_name = 'E001_2\\hazy.avi'
# vid_in_name = 'bedroom_0131\\hazy.avi'
# vid_size = (640, 480)
vid_size = (1280, 850)
vid_fps = 60
for g in [True, False]:
    for method in tqdm(['dcp', 'cap']):
        out_fold = os.path.join(_VIDEO_PATH, vid_in_name.split('\\')[0])
        out_fold = os.path.join(out_fold, 'dehazy')
        if not os.path.isdir(out_fold):
            os.mkdir(out_fold)
        vid_in = os.path.join(_VIDEO_PATH,vid_in_name)
        if g:
            vid_out = os.path.join(out_fold, f'{method}_gatm.avi')
        else:
            vid_out = os.path.join(out_fold, f'{method}.avi')
        time = process_video(vid_in, vid_out, method, vid_fps, vid_size, use_global = g)
        print(f'Processing time by {method}: {np.sum(time):.3f}')

 50%|████████████████████████████████                                | 1/2 [00:29<00:29, 29.59s/it]

'NoneType' object has no attribute 'copy'
Processing time by dcp: 27.654


100%|████████████████████████████████████████████████████████████████| 2/2 [01:23<00:00, 41.74s/it]


OpenCV(4.5.3) C:\Users\runneradmin\AppData\Local\Temp\pip-req-build-c2l3r8zm\opencv\modules\imgproc\src\color.cpp:182: error: (-215:Assertion failed) !_src.empty() in function 'cv::cvtColor'

Processing time by cap: 52.268


 50%|████████████████████████████████                                | 1/2 [00:29<00:29, 29.29s/it]

'NoneType' object has no attribute 'copy'
Processing time by dcp: 27.584


100%|████████████████████████████████████████████████████████████████| 2/2 [01:21<00:00, 40.93s/it]

OpenCV(4.5.3) C:\Users\runneradmin\AppData\Local\Temp\pip-req-build-c2l3r8zm\opencv\modules\imgproc\src\color.cpp:182: error: (-215:Assertion failed) !_src.empty() in function 'cv::cvtColor'

Processing time by cap: 50.951
